<a href="https://colab.research.google.com/github/ShawnLiu119/NLP_Bank_Complaint_Classification/blob/main/CustomerComplaint_Classification_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text Classification - Customer Complaints with LLM**

Purpose: explore the effectiveness of LLMs in classifying customer complaints and compare with traditional ML apporach

1. data source - https://www.kaggle.com/datasets/venkatasubramanian/automatic-ticket-classification


### Theory Walkthrough
Reference:https://towardsdatascience.com/fine-tune-smaller-transformer-models-text-classification-77cbbd3bf02b

A **decoder model** takes in a smaller input and outputs a larger text. GPT, which introduced impressive text generation back when, is a decoder model.

While larger language models offer more nuanced capabilities today, decoders were not built for tasks that involve extraction and labeling. For these tasks, we can use **encoder models**, which take in more input and provide a condensed output.

User Case for Encoder model:<br>
sentiment analysis, categorization, named entity recognition, and keyword extraction, among others.

In [ ]:
U